In [ ]:
import covasim as cv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import sciris as sc
import datetime as dt
import matplotlib.ticker as ticker
from matplotlib import rc
#%config InlineBackend.figure_formats = ['svg']
rc('font', **{'family': 'serif', 'serif': ['Times']})
rc('text', usetex=False)
rc('grid', linestyle="--", color='black')

# original sims
ttq_sims = cv.load('../fig5_projections/fig5.msims')
# updated sims
abc_sims = cv.load('../fig5_projections/fig5-abc.msims')
# data
refsim = abc_sims[0].sims[0] # A reference simulation, any will do
xlims = refsim.day('2020-01-27', '2020-08-31')



tsfn = './kc_data/20200614chop5_KingCounty_Covasim_extended.xlsx' # Time series data
tsdf = pd.read_excel(tsfn, engine='openpyxl')
tsdf['date'] = pd.to_datetime(tsdf['date']).dt.date

def roll(data):
    ''' Rolling window smoothing '''
    if hasattr(data, 'values'):
        data = data.values
    output = pd.Series(data).rolling(window=7).mean()
    return output

@ticker.FuncFormatter
def date_formatter(x,pos):
        # print(x)
    return (refsim['start_day'] + dt.timedelta(days=x)).strftime('%b-%d')

fig, axs = plt.subplots(nrows=2,sharex=True)

colors = ['#1b9e77', '#d95f02']
outputs = ['n_infectious','new_diagnoses']
for i, key in enumerate(outputs):
    ax = axs[i]
    # ttq
    res = ttq_sims['high'].results
    date = refsim.day(res['date']) # Convert to days
    best = res[key].values
    low  = res[key].low
    high = res[key].high
    ax.fill_between(date, roll(low), roll(high), facecolor=colors[0],alpha=0.4)
    ax.plot(date, roll(best), c=colors[0], alpha=1.0, label='TTQ')

    # ABC
    res = abc_sims['high'].results
    date = refsim.day(res['date']) # Convert to days
    best = res[key].values
    low  = res[key].low
    high = res[key].high
    ax.fill_between(date, roll(low), roll(high), facecolor=colors[1],alpha=0.4)
    ax.plot(date, roll(best), c=colors[1], alpha=1.0, label='ABC')

    ax.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax.get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

    if i==0:
        # repeat for status quo
        # ttq
        res = ttq_sims['actual'].results
        date = refsim.day(res['date']) # Convert to days
        best = res[key].values
        low  = res[key].low
        high = res[key].high
        ax.fill_between(date, roll(low), roll(high), facecolor=colors[0],alpha=0.4)
        ax.plot(date, roll(best), c=colors[0], alpha=1.0, label='TTQ (Status Quo)',linestyle='dashed')

        # ABC
        res = abc_sims['actual'].results
        date = refsim.day(res['date']) # Convert to days
        best = res[key].values
        low  = res[key].low
        high = res[key].high
        ax.fill_between(date, roll(low), roll(high), facecolor=colors[1],alpha=0.4)
        ax.plot(date, roll(best), c=colors[1], alpha=1.0, label='ABC (Status Quo)',linestyle='dashed')

        
        ax.set_xlim(xlims)
        
        ax.legend(frameon=True,edgecolor='black')
        day_stride = 21
        xmin,xmax = ax.get_xlim()
        ax.set_xticks(np.arange(xmin, xmax, day_stride))
        ax.xaxis.set_major_formatter(date_formatter)
        ax.legend(frameon=True,edgecolor='black',loc='upper right')
        sc.boxoff(ax=ax)
    ax.set_ylabel(key.replace('_',' ').title())

# plot SCAN data
# prevalence data
if 'n_infectious' in outputs:
    pop_size = 2.25e6
    scan_file = '../inputs/scanprev_5_21.csv'
    scan = pd.read_csv(scan_file)
    for i,r in scan.iterrows():
        label = "Data" if i==0 else None
        ts = np.mean(refsim.day(r['since'], r['to']))
        low  = r['lower']*pop_size
        high = r['upper']*pop_size
        mean = r['mean']*pop_size
        axs[0].plot([ts]*2, [low, high], alpha=1.0, color='k', zorder=1000)
        axs[0].plot(ts, mean, 'o', markersize=7, color='k', alpha=0.5, label=label, zorder=1000)

# Date
ax = axs[1]
date = refsim.day(tsdf['date'].tolist())
ax.plot(date, roll(tsdf['new_diagnoses']), label='Data', color='black')
ax.legend(frameon=True,edgecolor='black',loc='upper right')





# plot intervention
may31 = refsim.day('2020-05-31')
ax.axvline(may31, c='black', linestyle='--', alpha=0.4, lw=3)
yl = ax.get_ylim()
labely = yl[1]*1.03
ax.text(may31, labely, 'Stay-at-home lifted', color='black', alpha=0.9, style='italic', horizontalalignment='center')
ax.set_xlabel('Date')    

axs[1].set_ylabel('New Diagnoses')
axs[0].set_ylabel('Active Infections')

fig.tight_layout()
fig.savefig('../hetGPy-calibration/figs/ttq-abc-policy-comparison.pdf')